# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain & Pinecone Vector DB

> *This notebook should work well with the **`Data Science 3.0` (Python 3.10)** kernel in SageMaker Studio*

### Context
Previously we saw the model tell us how to change a tire, however, we had to provide it with additional contextual information before it would give us an accurate response. Then we explored an approach that leverages the models available with Bedrock and asked questions based on the model's training data as well as providing additional context. While that approach worked for short documents, it fails to scale to enterprise level question-answering where there could be large documents that cannot fit into the model's context window. 

### Pattern
We can improve upon this process by implementing an architecture called Retrieval Augmented Generation (RAG). RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant data in context. 

In this notebook we explain how to use the Question Answering pattern to find and leverage the documents that help answer our user's questions.

### Challenges
- How to manage large document(s) that exceed the token limit
- How to find the document(s) relevant to the question being asked

### Proposal
To the above challenges, this notebook proposes the following strategy:
#### Prepare documents
<img src="../../imgs/Embeddings_pinecone_lang.png" width="800"/>

Before answering the questions, the documents must be processed and a stored in a vector database
- Load the documents
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings
#### Ask question
<img src="../../imgs//53-rag-with-pinecone.png" width="800"/>

Once the document index is prepared, you can start asking questions. The relevant documents will be retrieved from the vector store based on the user's question. 
The following steps will be executed:
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks from the database
- Add those chunks to the context in the prompt
- Send the prompt to the model
- Get the contextual answer based on the documents retrieved

## Use case
#### Dataset
To demonstrate this architectural pattern we will be using the Amazon shareholder letters from the last few years.

## Implementation
This notebook is using the LangChain framework. LangChain integrates with different services and tools which make building RAG applications relatively easy. 
We will be using the following tools in this notebook:

- **LLM (Large Language Model)**: Anthropic Claude available through Amazon Bedrock

  This model will be used to interpret the document chunks and answer questions in a friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the documents
- **Document Loader**: PDF Loader available through LangChain

  This is the loader that loads the documents from a source. For the sake of this notebook, we are loading files from a local path. This could easily be replaced with a loader that loads documents from enterprise datastore.

- **Vector Store**: Pinecone Vector Database Free Tier available through pinecone.io.

  In this notebook we are using Pinecone to store both the embeddings and the documents. In an enterprise context this could be replaced with another persistent store such as AWS OpenSearch, RDS Postgres with pgVector, ChromaDB, Pinecone or Weaviate.
- **Index**: VectorIndex

  The index compares the input embedding and the document embeddings to find the relevant documents
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

Built with the help of ideas in this [notebook](https://www.pinecone.io/learn/series/langchain/langchain-retrieval-augmentation/) and this [notebook](01_qa_w_rag_claude.ipynb)

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In this notebook, we'll also need some extra dependencies:

- [Pinecone](http://pinecone.io), to store vector embeddings
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files

⚠️ **You would need your Pincone API key to proceed**

In [ ]:
%pip install -U \
    pinecone-client==2.2.4 \
    apache-beam==2.50.0 \
    # datasets==2.14.5 \
    tiktoken==0.4.0 --force-reinstall --quiet

In [ ]:
%pip install pydantic==1.10.13 --force-reinstall --quiet

In [ ]:
%pip install langchain>=0.1.11 langchain-aws==0.1.0
%pip install pypdf==4.1.0
%pip install langchain-community faiss-cpu==1.8.0 tiktoken==0.6.0 sqlalchemy==2.0.28

In [ ]:
import json
import os
import sys
import boto3
import botocore

bedrock_client = boto3.client('bedrock-runtime')

## Configure langchain

We begin by instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embeddings.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="...")`

In [ ]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain_aws import ChatBedrock

# - create the Anthropic Model
llm = ChatBedrock(
    model_id="anthropic.claude-v2", 
    # client=bedrock_client, 
    # model_kwargs={"max_tokens_to_sample": 200}
)
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_client)

## Data Preparation

In [ ]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

As part of Amazon's culture, the CEO always includes a copy of the 1997 Letter to Shareholders with every new release. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 1997 letter (last 3 pages) and overwrite them as processed files.

In [ ]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()

After downloading the files, we can load the documents with the help of [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM's context windows. Also, the embeddings model has a limit of 512 input tokens, which roughly translates to ~2000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [ ]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]
        
    print(f'{len(document)} {document}\n')
    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)

docs = text_splitter.split_documents(documents)

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')
print(f'After the split we have {len(docs)} documents as opposed to the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_doc_length(docs)} characters.')

In [ ]:
docs[0]

In [ ]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

A similar embeddings pattern could be used for an entire corpus and stored in a vector store.

This can be easily done using [Pinecone](https://python.langchain.com/docs/integrations/vectorstores/pinecone) implementation inside [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html). It uses the embeddings model along with the documents to create the entire vector store. Using the Index Wrapper we can abstract away most of the heavy lifting such as creating the prompt, getting the embeddings for the query, sampling the relevant documents, and calling the LLM. [VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation) helps us with that.


In [ ]:
from pinecone import Pinecone, ServerlessSpec
import time
import os
from tqdm.autonotebook import tqdm

# add Pinecone API key from app.pinecone.io
os.environ['PINECONE_API_KEY']="YOUR_API_KEY"
# set Pinecone environment - find next to API key in console
pc = Pinecone()
# add index name from pinecone.io
index_name = "bedrock-workshop"

if index_name in pc.list_indexes():
    pc.delete_index(index_name)

pc.create_index(name=index_name, dimension=sample_embedding.shape[0], metric="dotproduct", spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
))
# wait for index to finish initialization
while not pc.describe_index(index_name).status["ready"]:
    time.sleep(1)

In [ ]:
index = pc.Index(index_name)
index.describe_index_stats()

**⚠️⚠️⚠️ NOTE: it might take few minutes to run the following cell ⚠️⚠️⚠️**

In [ ]:
%pip install langchain-pinecone --quiet

In [ ]:
%%time
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(docs, index_name=index_name, embedding=bedrock_embeddings)

In [ ]:
index.describe_index_stats()

## LangChain Vector Store and Querying

We construct our index independently of LangChain. That’s because it’s a straightforward process, and it is faster to do this with the Pinecone client directly. However, we’re about to jump back into LangChain, so we should reconnect to our index via the LangChain library.

In [ ]:
# text_field = "text"

# switch back to normal index for langchain

vectorstore = PineconeVectorStore(index_name=index_name, embedding=bedrock_embeddings)


#### We can use the similarity search method to make a query directly and return the chunks of text without any LLM generating the response.

In [ ]:
query = "How has AWS evolved?"

vectorstore.similarity_search(query, k=3)  # our search query  # return 3 most relevant docs

#### All of these are relevant results, telling us that the retrieval component of our systems is functioning. The next step is add our LLM so it can answer our question based on the information retrieved from these queries to the Pinecone database.

## Generative Question Answering

In generative question-answering (GQA), we pass our question to the Claude-2 model but instruct it to base the answer on the information returned from Pinecone. We can do this in LangChain by using the RetrievalQA chain.

In [ ]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
prompt_template = """Human: Use the following pieces of context to provide a concise answer in Italian to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context>

<question>
{question}
</question>


Assistant:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": PROMPT})

#### Let’s try this with our earlier query:

In [ ]:
qa.invoke(query)

### The response we get this time is generated by our LLM based on the retrieved information from our vector database.

We’re still not entirely protected from convincing yet false hallucinations by the model. They can happen from time to time, and it’s unlikely that we can eliminate them completely. However, we can do more to improve our trust in the answers provided.

An effective way of doing this is by adding citations to the response, allowing a user to see where the information is coming from. We can do this using a slightly different version of the RetrievalQA chain called RetrievalQAWithSourcesChain.

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), return_source_documents=True)

In [ ]:
qa_with_sources.invoke(query)

Now we have answered the question being asked but also included the source of this information being used by the LLM.

We’ve learned how to ground Large Language Models with source knowledge by using a vector database as our knowledge base. Using this, we can encourage accuracy in our LLM’s responses, keep source knowledge up to date, and improve trust in our system by providing citations with every answer.

We can use this embedding of the query to then fetch relevant documents.
Now our query is represented as embeddings we can do a similarity search of our query against our data store providing us with the most relevant information.

### Customizable option
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the helpf of [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) which can be specific to the model.

Note: In this example we are using Anthropic Claude as the LLM under Amazon Bedrock, this particular model performs best if the inputs are provided under `Human:` and the model is requested to generate an output after `Assistant:`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
)
result = qa.invoke({"query": query})
print(result["result"])

In [ ]:
result["source_documents"]

## Conclusion
Congratulations on completing this module on retrieval augmented generation! This is an important technique that combines the power of large language models with the precision of retrieval methods. By augmenting generation with relevant retrieved examples, the responses we received become more coherent, consistent and grounded. You should feel proud of learning this innovative approach. I'm sure the knowledge you've gained will be very useful for building creative and engaging language generation systems. Well done!

In the above implementation of RAG based Question Answering we have explored the following concepts and how to implement them using Amazon Bedrock and its LangChain integration.

- Loading documents and generating embeddings to create a vector store
- Retrieving documents to the question
- Preparing a prompt which goes as input to the LLM
- Present an answer in a human friendly manner
- keep source knowledge up to date, and improve trust in our system by providing citations with every answer.

### Take-aways
- Experiment with different Vector Stores
- Leverage various models available under Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Integration with enterprise data stores

### Clean Up
Delete the Pinecone index

In [ ]:
pc.delete_index(name=index_name)